In [1]:
using Pkg
Pkg.activate("D:\\Science\\Code\\Julia\\envr")

using StaticArrays, DifferentialEquations, DynamicalSystems, JLD

  Activating project at `D:\Science\Code\Julia\envr`


In [3]:
function TM(u, p, t)

    U(y) = p[8] + p[9] / ( 1.0 + exp( -50.0 * (y - p[7]) ) )
    σ(x) = 1.0 / ( 1.0 + exp( -20.0 * (x-p[6]) ) )

    du1 = (-u[1] + p[1] * log( 1.0 + exp( (p[5] * U(u[3]) * u[2] * u[1] + p[11]  ) / (p[1]) ) ) ) / p[2]
    du2 = (1.0 - u[2])/p[3] - U(u[3])*u[2]*u[1]
    du3 = (-u[3])/p[4] + p[10] * σ(u[2])
    return SA[du1, du2, du3]
end;

In [4]:
function output(idx_I0, idx_U0, I0, U0, u0)
    println("Индекс I0: $idx_I0; I0: $I0");flush(stdout)
    println("Индекс U0: $idx_U0; U0: $U0");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function output(idx_U0,U0, u0)
    println("Индекс U0: $idx_U0; U0: $U0");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function output_end_iter(ΛΛ, u0_lc)
    println("Спектр: $(ΛΛ)");flush(stdout)
    println("Последняя точка: $(u0_lc)");flush(stdout)
end
function separate()
    println();flush(stdout)
    println("---------");flush(stdout)
end;

In [10]:
function init_ds(U0_, u0_lc)
    p = SA[α, τ, τD, τy, J, xthr, ythr, U0_, ΔU0, β, I0_start]
    ds = ContinuousDynamicalSystem(TM, u0_lc, p)
    return ds
end
function init_ds(U0_, I0_, u0_lc)
    p = SA[α, τ, τD, τy, J, xthr, ythr, U0_, ΔU0, β, I0_]
    ds = ContinuousDynamicalSystem(TM, u0_lc, p)
    return ds
end
function goto_attractor(ds_, time_attract)
    tr = trajectory(ds_, time_attract; Ttr = tt, diffeq = integ_set)
    u0_lc = tr[end]
    return u0_lc
end
function spectrum(ds_)
    ΛΛ = lyapunovspectrum(ds_, t; diffeq = integ_set)
    return ΛΛ
end
function save_output(idx_U0, ΛΛ, u0_lc)
    Λs[1, idx_U0, :] = ΛΛ
    u0s[1, idx_U0, :] = u0_lc
end
function save_output(index_I0, idx_U0, ΛΛ, u0_lc)
    Λs[index_I0, idx_U0, :] = ΛΛ
    u0s[index_I0, idx_U0, :] = u0_lc
end
function save_tofile()
    save("LSs_200x200.jld", "data", Λs)
    save("u0s_200x200.jld", "data", u0s)
end

save_tofile (generic function with 1 method)

In [5]:
t = 1000.0; tt = 250.0; tstep = 0.001
time_attract = 100.0
integ_set = (alg = RK4(), adaptive = false, dt = tstep);

In [6]:
const τ = 0.013;  const τD = 0.080;  const τy = 3.3;  const J = 3.07;  const β = 0.300
const xthr = 0.75; const ythr = 0.4
const α = 1.58;  U0 = 0.3;  const ΔU0 = 0.305; I0 = -1.40;

In [7]:
u0 = SA[8.39353, 0.73803, 0.429055];

3-element SVector{3, Float64} with indices SOneTo(3):
 8.39353
 0.73803
 0.429055

In [8]:
function det_step(start, finish, len)
    return (finish - start) / len
end

det_step (generic function with 1 method)

In [97]:
general_len = 400

I0_start = -1.40; I0_finish = -1.709; len_I0 = general_len
U0_start = 0.30; U0_finish = 0.47; len_U0 = general_len

step_I0 = det_step(I0_start, I0_finish, len_I0)
step_U0 = det_step(U0_start, U0_finish, len_U0)

I0range = range( I0_start, I0_finish, length = len_I0 )
U0range = range( U0_start,  U0_finish, length = len_U0)

println("My version step: ", step_I0, "  ", step_U0)
println("Julia version step: ", I0range.step.hi, "  ", U0range.step.hi)

My version step: -0.0007725000000000004  0.000425
Julia version step: -0.0007744360902255387  0.00042606516290721874


In [9]:
Λs = zeros(len_I0, len_U0, 3)
u0s = zeros(len_I0, len_U0, 3);

In [11]:
for (idx_U0, U0_) in enumerate(U0range)
    
    if idx_U0 == 1 # Проверка на первую итерацию
        # Если использовать while условие не нужно!
        global u0_lc = u0
    end
    
    # output(idx_U0,U0_, u0_lc)
    
    ds = init_ds(U0_, u0_lc) # Инициализация системы
    u0_lc = goto_attractor(ds, time_attract) # Выход на аттрактор
    ds = init_ds(U0_, u0_lc)
    
    ΛΛ = spectrum(ds) # Вычисление спектра
    
    # output_end_iter(ΛΛ, u0_lc)
    
    save_output(idx_U0, ΛΛ, u0_lc) # Сохранение значений
    save_tofile()
    
    # separate()

end

In [ ]:
for (idx_U0, U0_) in enumerate(U0range)
    for (idx_I0, I0_) in enumerate(I0range)
        
        if idx_I0 == 1 # Если использовать while условие не нужно!
            continue
        end
        
        
        u0_lc = u0s[idx_I0 - 1, idx_U0, :]
        
        # output(idx_I0, idx_U0, I0_, U0_, u0_lc)
        
        ds = init_ds(U0_, I0_, u0_lc)
        u0_lc = goto_attractor(ds, time_attract) # Выход на аттрактор
        ds = init_ds(U0_, I0_, u0_lc) 
        ΛΛ = spectrum(ds) # Вычисление спектра
        
        # output_end_iter(ΛΛ, u0_lc)
        
        save_output(idx_I0, idx_U0, ΛΛ, u0_lc)
        save_tofile()
        
        # separate()
        
    end
end